# Bike Thefts Berlin - EDA

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv('../data/Fahrraddiebstahl.csv', index_col=0, parse_dates=True, encoding = 'unicode_escape')

In [5]:
df.isna().sum()

TATZEIT_ANFANG_DATUM     0
TATZEIT_ANFANG_STUNDE    0
TATZEIT_ENDE_DATUM       0
TATZEIT_ENDE_STUNDE      0
LOR                      0
SCHADENSHOEHE            0
VERSUCH                  0
ART_DES_FAHRRADS         0
DELIKT                   0
ERFASSUNGSGRUND          0
dtype: int64

In [6]:
df['DELIKT'].unique()

array(['Fahrraddiebstahl', 'Keller- und Bodeneinbruch'], dtype=object)

## Inspecting features

In [7]:
def time_parser(df, time_parsables):
    '''Parses columns encoded as strings to datetime-objects.'''
    for col in time_parsables:
        df[col] = pd.to_datetime(df[col])
    return df

In [8]:
def check_unique_vals(df):
    '''Checks unique values per column.'''
    for col in df.columns:
        print(df[col].unique())

In [9]:
check_unique_vals(df)

['18.02.2023' '17.02.2023' '19.02.2023' '16.02.2023' '12.02.2023'
 '13.02.2023' '15.02.2023' '10.02.2023' '11.02.2023' '27.01.2023'
 '17.02.2022' '12.01.2023' '14.02.2023' '06.01.2023' '24.01.2023'
 '19.12.2022' '08.02.2023' '06.02.2023' '15.01.2023' '12.08.2022'
 '01.02.2023' '26.01.2023' '28.01.2023' '09.02.2023' '07.02.2023'
 '10.12.2022' '10.01.2023' '03.02.2023' '31.01.2023' '17.01.2023'
 '22.01.2023' '05.12.2022' '20.01.2023' '29.12.2022' '10.10.2022'
 '25.01.2023' '10.11.2022' '05.02.2023' '01.01.2023' '19.11.2022'
 '04.02.2023' '07.02.2022' '21.01.2023' '23.01.2023' '02.02.2023'
 '17.11.2022' '30.01.2023' '04.01.2023' '11.01.2023' '19.01.2023'
 '09.12.2022' '29.01.2023' '01.02.2022' '14.01.2023' '13.01.2023'
 '16.01.2023' '18.11.2022' '22.09.2022' '28.12.2022' '18.01.2023'
 '12.12.2022' '03.01.2023' '31.07.2022' '20.12.2022' '08.01.2023'
 '24.01.2022' '19.10.2022' '08.12.2022' '23.01.2022' '07.01.2023'
 '21.01.2022' '04.12.2022' '27.12.2022' '27.10.2022' '16.12.2022'
 '26.12.20

In [ ]:
df.head(3)

In [ ]:
time_parsable = ['TATZEIT_ANFANG_DATUM', 'TATZEIT_ENDE_DATUM', 'TATZEIT_ANFANG_STUNDE', 'TATZEIT_ANFANG_STUNDE']

Plan:
- parse relevant time-related cols to datetime so that you can calculate time periods. Done
- group by: district and plot n(thefts). Done
- plot thru time
- check distributions
- check correlations
- check for time gaps, and interpolate them

In [ ]:
df_time_parsed = time_parser(df, time_parsable)

In [ ]:
df_time_parsed["Year"] = df_time_parsed.index.year
df_time_parsed["Month"] = df_time_parsed.index.month

In [ ]:
df_time_parsed.head()

In [ ]:
y = df_time_parsed['SCHADENSHOEHE']

In [ ]:
p = df_time_parsed.groupby('LOR')['SCHADENSHOEHE'].mean().hist()
plt.xticks(rotation=45)
p;

## EDA - features

In [ ]:
p4 = df.groupby(["Year","Month"])["n(Loans)_Outstanding_Other"].mean().plot() 
plt.xticks(rotation=45)
p4

### The number of transactions of SME clients of the SB with other institutions have decreased strongly since the year 2020.
* in the year 2021 these transactions skyrocket. Maintained outliers. 

#### EDA - amount of last disbursements (at each cut-off date) => "monetary"

In [ ]:
p5 = df.groupby(["Year","Month"])["Amount_Last_Disbursement"].mean().plot() 
plt.xticks(rotation=45)
p5

### During the time period between 2018 and May 2021 the amounts disbursed to SME clients tended to decrease.
* trend changes in april 2021. Maintained outliers. 

#### EDA - total accumulated interest since last disbursement, per client (at each cut-off date) => "monetary"

In [ ]:
p6 = df.groupby(["Year", "Month"])["Total_Accumulated_Interest_per_Client"].mean().plot()
plt.xticks(rotation=45)
p6

### Accumulated interest skyrockets at the beginning of 2020 and decreases sharply in the course of the year to pre 2020 levels.
* in the year 2021 interest accumulated skyrockets again. Maintained outliers. 

In [ ]:
# drop time-related cols again
df.drop(["Year", "Month"], axis = 1, inplace = True)

#### EDA - socioeconomic features 

In [ ]:
(p9.ggplot(data=df,
           mapping=p9.aes(x="Client_Age_Years")
                       )
    + p9.geom_bar(
       alpha=0.4,
        color='#F6F6F6', 
        fill='#0C475B')
    + p9.facet_wrap("Client_Status_Post3Months")
)

Result not intuitive, since the majority group are customers that do not churn. I conclude that churn has been encoded wrongly. Hence, re-enconding churn as "1":

In [ ]:
df["Client_Status_Post3Months"].unique

In [ ]:
df["Client_Status_Post3Months"] = df["Client_Status_Post3Months"].replace({0:1, 1:0})

In [ ]:
df["Client_Status_Post3Months"].unique

In [ ]:
(p9.ggplot(data=df,
           mapping=p9.aes(x="Client_Age_Years")
                       )
    + p9.geom_bar(
       alpha=0.4,
        color='#F6F6F6', 
        fill='#0C475B')
    + p9.facet_wrap("Client_Status_Post3Months")
)

Now it makes sense: distribution of minority group shows less counts compared to majority group.

In [ ]:
(p9.ggplot(data=df,
          mapping=p9.aes(x="Client_Sex")
                       )
    + p9.geom_bar(
       alpha=0.4,
       color='#F6F6F6', 
        fill='#0C475B')
    + p9.facet_wrap("Client_Status_Post3Months")
)

Gender has been encoded as: "1" women...resulting plot makes sense, since proportion of male clients is higher in practice.

In [ ]:
(p9.ggplot(data=df,
          mapping=p9.aes(x="Resident_Capital")
                       )
    + p9.geom_bar(
       alpha=0.4,
       color='#F6F6F6', 
       fill='#0C475B')
    + p9.facet_wrap("Client_Status_Post3Months")
)

Intuitive: 
- minority group has less counts than majority group
- for both groups: the majority of counts are masculine customers
- for both genders: most of the customers in the portfolio are clients that actually live in the capital city


In [ ]:
(p9.ggplot(data=df,
          mapping=p9.aes(x="Client_Married")
                       )
   + p9.geom_bar(
       alpha=0.4,
       color='#F6F6F6', 
       fill='#0C475B')
    + p9.facet_wrap("Client_Status_Post3Months")
)

Intuitive: 
- same interpretation as prior bar plot - only re marital status of customers.

## EDA - correlation between features and target ("client status")

In [ ]:
corr = df.corr()

mask = np.triu(np.ones_like(corr, dtype=bool))

f, ax = plt.subplots(figsize=(8, 6))

cmap = sns.diverging_palette(230, 20, as_cmap=True)

sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
             square=True, linewidths=.5, cbar_kws={"shrink": .5})

"Zooming in"...

In [ ]:
df_RFM = df.drop(["Resident_Capital", "Client_Age_Years","Client_Sex", "Client_Married","LO_Active_Employee_Post3Months",
                 "LO_Active_Employee_Post6Months", "LO_Active_Employee_Prior3Months", "LO_Active_Employee_Prior6Months",
                 "LO_Changed", "n(Months)_Change_LO"], axis = 1)

In [ ]:
corr_RFM = df_RFM.corr()

mask = np.triu(np.ones_like(corr_RFM, dtype=bool))

f, ax = plt.subplots(figsize=(8, 6))

cmap = sns.diverging_palette(230, 20, as_cmap=True)

sns.heatmap(corr_RFM, mask=mask, cmap=cmap, vmax=.3, center=0,
             square=True, linewidths=.5, cbar_kws={"shrink": .5})

Socioeconomic variables and target variable do not show high correlations.

In [ ]:
df_RFM = df_LO = df.drop(["Resident_Capital", "Client_Age_Years","Client_Sex", "Client_Married","Amount_Last_Disbursement",
                 "n(Months)_Since_Last_Disbursement", "n(Loans)_Outstanding_Maynas", "n(Loans)_Outstanding_Other",
                 "n(Additional_Loans)_Post3Months", "n(Additional_Loans)_Post6Months", "Total_Accumulated_Interest_per_Client",
                 "n(Additional_Loans)_Pre3Months", "n(Additional_Loans)_Pre6Months"]      
                 , axis = 1)

In [ ]:
corr_LO = df_LO.corr()

mask = np.triu(np.ones_like(corr_LO, dtype=bool))

f, ax = plt.subplots(figsize=(8, 6))

cmap = sns.diverging_palette(230, 20, as_cmap=True)

sns.heatmap(corr_LO, mask=mask, cmap=cmap, vmax=.3, center=0,
             square=True, linewidths=.5, cbar_kws={"shrink": .5})

### Main correlations between features and the target variable are found with those variables that describe relationship between clients and their Loan Officers!

## Checking for class imbalance

In [ ]:
df.info()

In [ ]:
y = df["Client_Status_Post3Months"]

In [ ]:
(y == 1).sum(), (y == 0).sum() 

In [ ]:
y.value_counts()/df.shape[0]

5.624 counts of clients who have left the SB after 3 months (for each cut-off date) <=>  37% of all clients. This is problematic:
- based on a separate analysis, one concludes that monthly average in the established time period is approx. 2%. 37% for churn simply is unrealistic.
- while extracting data from database, most probably some registers were duplicated manually. If so  => extract them.

## Checking for duplicates

In [ ]:
df2 = df.groupby(["Client_Status_Post3Months"]).agg({"Client_ID" : "nunique", "Resident_Capital" : "count"})
df2 = df2.reset_index()
df2 = df2.rename(columns={"Client_ID" : "Unique_Frequency", "Resident_Capital" : "Frequency"})
df2

In [ ]:
check_duplicates = np.array(df2)
check_duplicates

In [ ]:
prop_duplicates = check_duplicates[0,1]/check_duplicates[0,2]*100
prop_duplicates

Confirming high number of duplicates. Need to extract them.

In [ ]:
df_unique = df.drop_duplicates(subset="Client_ID", keep="first") 

In [ ]:
df_unique.info()

=> data set reduced by one third after dropping duplicates!

In [ ]:
df_unique.shape

In [ ]:
y = df_unique["Client_Status_Post3Months"]

In [ ]:
y.value_counts()/df_unique.shape[0]

Unintuitive: data set is not imbalanced. Can't achieve extraction of duplicates. However => continue analysis with this data set.

Preparing dataset for training models:

In [ ]:
df_unique.head()

In [ ]:
df_unique.shape

In [ ]:
df_unique.to_csv("Tabla_01_English_Unique_postEDA.csv")